<center><h2>Multi-arm Bandits Algorithms</h2></center>


<center><img src="http://i1.wp.com/banditalgs.com/wp-content/uploads/2016/09/cropped-bandit-algorithm-full.png?fit=512%2C512" width="75%"/></center>

<center><h2>Learning Outcomes</h2></center>

Explain and code the following multi-arm bandits algorithms:

1. Random
1. Epsilon-Greedy Algorithm  
2. Softmax  
4. Bayesian Bandit 

<center><h2>Random Algorithm</h2></center>

1) Pure exploration.

2) Good for checking the entire system.

3) Reasonable baseline.

<center><h2>Let's Code a Bandit</h2></center>

In [103]:
reset -fs

In [104]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import warnings
warnings.filterwarnings('ignore')

palette = "Dark2"
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
# Import common packages
import math
import random

import numpy as np
import scipy.stats as stats

# Seed the random number generators so you get the same results every time.
np.random.seed(42)
random.seed(42)

In [106]:
# Bandit is the class that setups and runs our multi-arm bandit
from bandit import Bandit

<center><h2>Student Activity</h2></center>

Implement the following 

```python
def random_choice(self): 
    "Pick a bandit uniformly at random."
    pass
    
```

In [143]:
def random_choice(self): 
    "Pick a bandit uniformly at random."
    return np.random.randint(low=0, high=self.n_bandits)

In [108]:
def random_choice(self): 
    "Pick a bandit uniformly at random."
    return np.random.randint(low=0, high=self.n_bandits) # Return the index of the winning bandit.

<center><h2>Check for understanding</h2></center>

Why use `np.random.randint` instead of `random.choice`?

`random.choice` is much slower because it selects among any options.

`np.random.randint` is optimitize to just select integers (the index of the arm in this case)

In [149]:
# Let's instantiate the class and simulate outcomes
b = Bandit(p_array=[0.6, 0.6, 0.6, 0.6],
           choice_function=random_choice)
b.sample_bandits(n=1_000)
# Let's see which arm is the winner
b.max_mean()

2

In [110]:
from bandit import print_results

print_results(b)

Options: 	       A      B      C      D
Probablities: 	    0.60   0.60   0.60   0.60
Number of wins:      165    155    132    147
Number of trials:    263    258    243    236
Conversion rates:   0.63   0.60   0.54   0.62

A total of 599 wins out of 1,000 trials.


<center><img src="images/eg.jpg" width="55%"/></center>

Some percent of the time we explore - randomly choose one of the options. 

The rest of the time we exploit - choose the option that has so far had the highest conversion rate. 

<center><h2>Epsilon-Greedy Algorithm</h2></center>
<br>
<center><img src="images/epislon_greedy.png" width="75%"/></center>

a is action.  
t is time.  
$a^*$ is optimal action.  
ε epsilon a parameter to choose is the probability that we explore.  

<center><h2>Check for understanding</h2></center>

What is the range for epsilon?

Epsilon is a probability, thus bounded between 0 and 1.

<center><h2>Epsilon Default Value</h2></center>
<br>
<br>
<center>.1 (or 10% exploration)</center>

<center><h2>Student Activity</h2></center>

Implement the following 

```python
def epsilon_greedy(self, epsilon=0.1):
    """Pick a bandit uniformly at random epsilon percent of the time.
    Otherwise pick the bandit with the best observed proportion of winning.
    Return the index of the winning bandit."""
    pass
```               

Write the code without looking at other people's implementation.   
It is okay to look at course notes and library documentation.

In [150]:
def epsilon_greedy(self, epsilon=0.1):
    """Pick a bandit uniformly at random epsilon percent of the time.
    Otherwise pick the bandit with the best observed proportion of winning.
    Return the index of the winning bandit."""
    if random.random() <= epsilon:
        return np.random.randint(low=0, high=self.n_bandits)
    else:
        return np.argmax(self.wins / (self.trials+1))

In [112]:
def epsilon_greedy(self, epsilon=0.1):
    """Pick a bandit uniformly at random epsilon percent of the time.
    Otherwise pick the bandit with the best observed proportion of winning.
    Return the index of the winning bandit."""
    if random.random() <= epsilon:
        return np.random.randint(low=0, high=self.n_bandits) # Return a random choice
    else:
        return np.argmax(self.wins / (self.trials + 1)) # Return the current best choice

In [113]:
# Let's instantiate the class and simulate outcomes
b = Bandit(p_array=[0.9, 0.6, 0.6, 0.6],
                    choice_function=epsilon_greedy)
b.sample_bandits(n=1_000)
# Let's see which arm is the winner
print_results(b)

Options: 	       A      B      C      D
Probablities: 	    0.90   0.60   0.60   0.60
Number of wins:      848     11      8      9
Number of trials:    936     23     16     25
Conversion rates:   0.91   0.48   0.50   0.36

A total of 876 wins out of 1,000 trials.


<center><h2>A Common Real-world Design Pattern for Epsilon-Greedy </h2></center>
    
1. At the beginning, set epsilon high (.15-.25).
1. After fix number of trials, lower (.05-.1).
1. Keep low and fixed for remaining trails.

This is similar to a fixed schedule for learning rate decay for stochastic gradient descent.

<center><h2>What are limitations of Epsilon-Greedy algorithm?</h2></center>

1) At the beginning, we might select exploitation but do __not__ yet know which version is right.

2) If epsilon is .10, the epsilon-greedy algorithm will eventually show the best option 90% of the time. 

So this means that 10% of the time the algorithm will continue to randomly show different versions of the site, no matter how confident we are that a certain version is the best. There is a point where we should stop the exploration.

<center><h2> Softmax Algorithm</h2></center>

Choose the arm in proportion to its estimated value.

Say $p_A$, $p_B$ and $p_C$ are the conversion rates for three versions of the site. We would choose site A with the following probability  

$$ \frac{exp(p_A/ \tau)}{exp(p_A/ \tau) + exp(p_B/ \tau) + exp(p_C/ \tau)} $$

$\tau$ is a chosen parameter that controls the ‘randomness’ of the choice, usually around 0.01.

<center><h2>Student Activity</h2></center>

Implement the following 

```python
def softmax(self, tau=0.01):
    """Pick an bandit according to the Boltzman Distribution.
    Return the index of the winning bandit."""
    pass
```                                 

In [114]:
def softmax(self, tau=0.01):
    """Pick an bandit according to the Boltzman Distribution.
    Return the index of the winning bandit."""

    # Make sure to play each bandit at least once
    if len(self.trials.nonzero()[0]) < self.n_bandits:
        return np.random.randint(0, self.n_bandits)

    # Compute the terms of the softmax function
    numerators = np.zeros(len(self.wins))
    denom = 0
    for i in range(len(self.wins)):
        numerators[i] = math.exp((self.wins[i] / (self.trials[i]+1)) / tau)
        denom += numerators[i]

    # Compute the "pmf" of the bandits
    last = 0
    softmax_vals = np.zeros(len(self.wins))
    for i in range(len(self.wins)):
        softmax_vals[i] = last + (numerators[i] / denom)
        last = softmax_vals[i]

    # Pick a random number and see where it falls in the range
    r = random.random()
    for i in range(len(self.wins)):
        if r <= softmax_vals[i]:
            return i

In [115]:
# Let's instantiate the class and simulate outcomes
b = Bandit(p_array=[0.9, 0.6, 0.6, 0.6],
                    choice_function=softmax)
b.sample_bandits(n=1_000)
# Let's see which arm is the winner
print_results(b)

Options: 	       A      B      C      D
Probablities: 	    0.90   0.60   0.60   0.60
Number of wins:        1    577      2      0
Number of trials:      2    994      3      1
Conversion rates:   0.50   0.58   0.67   0.00

A total of 580 wins out of 1,000 trials.


<center><h2>Softmax Limitation</h2></center>

It does __not__ take into account how many times an arm has been pulled

<center><h2>Bayesian Bandit (or Thompson Sampling)</h2></center>

1. Calculate Posterior for each "arm" / action
2. Pick a random sample from each Posterior
3. Choose the arm with highest reward
4. Update Posteriors for next round

<center><h2>Bayesian Bandit (or Thompson Sampling)</h2></center>

<br>
<center><img src="http://fastml.com/images/ab-testing/bandits_small.png" width="50%"/></center>




<center><a href="https://learnforeverlearn.com/bandits/">Demo!</a></center>



<center><h2>Thompson Sampling Steps</h2></center>

1. For each k, sample $θ_k ∼ π(θ_k | Dt)$
1. Then play bandit $k = arg max_k \hatθ_k$ 
1. Update played bandit win or lost

<center><h2>Thompson Sampling for Bernoulli Bandits</h2></center>

Given k bandits, each k pays off a reward of 1 with probability
$θ_k$ and 0 with probability 1 − $θ_k$. 

Model each bandit as $π$. Parameterized as $π(θ_k | D_t)$

Parameterization of the Beta family of distribution $θ ∼ Beta(α, β)$

Simple (but effective) Posterior is:

$$θ_k | D_t ∼ Beta(1 + n_{k0}, 1 + n_{k1})$$


Assume a uniform distribution is achieved using prior α = prior β = 1. 

- $\alpha = 1 + $ number of times bandit has won
- $\beta = 1 + $ number of times bandit has lost  

Another sensible defaults is to use an uninformative prior by setting prior_alpha = prior_beta = 0.5.


<center><h2>Full Bayesian updating</h2></center>

- Useful if you do not assume have stationary data
- However, it is far more complicated and much slower

Use the Bayesian Beta-Binomial conjugate prior techniques used to model the click-through rate as our base model for each of the bandits.  

Sample from each bandit’s distribution and play the bandit with the highest value.

It will naturally converge on the bandit with the best payout.  

[Source](http://fastml.com/ab-testing-with-bayesian-bandits-in-google-analytics/)

[Source](https://davidrosenberg.github.io/mlcourse/in-prep/thompson-sampling-bernoulli.pdf)

<center><h2>Student Activity</h2></center>

Implement the following 

```python
def bayesian_bandit(self):
    """Randomly sample from a beta distribution for each bandit and pick the one
    with the largest value.
    Return the index of the winning bandit."""    
    pass
```     

Hints: 

- Use `scipy.stats.beta`
- Seriously - use the all the methods on `scipy.stats.beta`

In [116]:
def bayesian_bandit(self):
    """Randomly sample from a beta distribution for each bandit and pick the one
    with the largest value.
    Return the index of the winning bandit."""    
    pass

In [117]:
def bayesian_bandit(self):
    """Randomly sample from a beta distribution for each bandit and pick the one
    with the largest value.
    Return the index of the winning bandit."""    
    
    # Make sure to play each bandit at least once
    if len(self.trials.nonzero()[0]) < self.n_bandits:
        return np.random.randint(0, self.n_bandits)

    # Create a beta distribution for each bandit and draw a random sample
    samples = np.zeros(len(self.wins))
    for i in range(len(self.wins)):
        samples[i] = stats.beta(1+self.wins[i], (1+self.trials[i]-self.wins[i])).rvs(1)
    return np.argmax(samples)

In [118]:
# Let's instantiate the class and simulate outcomes
b = Bandit(p_array=[0.9, 0.6, 0.6, 0.6],
                    choice_function=bayesian_bandit)
b.sample_bandits(n=1_000)
# Let's see which arm is the winner
print_results(b)

Options: 	       A      B      C      D
Probablities: 	    0.90   0.60   0.60   0.60
Number of wins:      872      4      5     11
Number of trials:    966      8      9     17
Conversion rates:   0.90   0.50   0.56   0.65

A total of 892 wins out of 1,000 trials.


<center><h2>Simulate Results With The Implementations</h2></center>

In [119]:
p_array = [0]*4 # No winners
# p_array = [1]*4 # All winners
# p_array = [.5]*4 # All the same
# p_array = [.01, .4, .8, .999] # All different

strategies = [epsilon_greedy, softmax, bayesian_bandit]

for strategy in strategies:
    b = Bandit(p_array=p_array,
                choice_function=strategy)
    b.sample_bandits(n=1_000) # 1000 is a good value
    print()
    print('#'*50)
    print()
    print(strategy.__name__.title())
    print_results(b)


##################################################

Epsilon_Greedy
Options: 	       A      B      C      D
Probablities: 	    0.00   0.00   0.00   0.00
Number of wins:        0      0      0      0
Number of trials:    926     28     25     21
Conversion rates:   0.00   0.00   0.00   0.00

A total of 0 wins out of 1,000 trials.

##################################################

Softmax
Options: 	       A      B      C      D
Probablities: 	    0.00   0.00   0.00   0.00
Number of wins:        0      0      0      0
Number of trials:    241    247    271    241
Conversion rates:   0.00   0.00   0.00   0.00

A total of 0 wins out of 1,000 trials.

##################################################

Bayesian_Bandit
Options: 	       A      B      C      D
Probablities: 	    0.00   0.00   0.00   0.00
Number of wins:        0      0      0      0
Number of trials:    246    249    250    255
Conversion rates:   0.00   0.00   0.00   0.00

A total of 0 wins out of 1,000 trials.


In [120]:
probs ={'atlantic_city': [0.1, 0.1, 0.1, 0.90],
        'las_vegas':     [0.1, 0.1, 0.1, 0.12],
        'reno':          [0.1, 0.2, 0.4, 0.50]}

for prob in  probs.items():
    p_array = prob[1]
    for strategy in strategies:
        b = Bandit(p_array=p_array,
                    choice_function=strategy)
        b.sample_bandits(n=1_000) # 1000 is a good value
        print()
        print('#'*50)
        print()
        print(strategy.__name__.title())
        print_results(b)


##################################################

Epsilon_Greedy
Options: 	       A      B      C      D
Probablities: 	    0.10   0.10   0.10   0.90
Number of wins:        7      2      3    743
Number of trials:    107     29     22    842
Conversion rates:   0.07   0.07   0.14   0.88

A total of 755 wins out of 1,000 trials.

##################################################

Softmax
Options: 	       A      B      C      D
Probablities: 	    0.10   0.10   0.10   0.90
Number of wins:        0      0      0    881
Number of trials:      3      1      1    995
Conversion rates:   0.00   0.00   0.00   0.89

A total of 881 wins out of 1,000 trials.

##################################################

Bayesian_Bandit
Options: 	       A      B      C      D
Probablities: 	    0.10   0.10   0.10   0.90
Number of wins:        0      0      0    882
Number of trials:      3      3      3    991
Conversion rates:   0.00   0.00   0.00   0.89

A total of 882 wins out of 1,000 trials.

######

<center><h2>Which MAB strategy should I use?</h2></center>



<center><img src="images/bandit_choice.png" width="100%"/></center>

<center><img src="images/map.jpeg" width="85%"/></center>

<center><h2>Challenge Question</h2></center>

Think about putting this into production. What kind of testing would you have to do (unit, integration, acceptance)? How would you monitor the different choices?

Summary
-----

- Epsilon-Greedy Algorithm sometimes pick randomly, other times pick the best.
- Softmax is structured exploration.
- Bayesian Bandit Algorithm models what we have seen (and uncertainty). Randomly sample, then pick best.

<center><h2>Bonus Materials</h2></center>

Which strategy should I use?
------
<center><img src="images/compare.png" width="900"/></center>

3) Upper Confidence Bound (UCB)
-----

Choose the strategy that where the following value is the largest:  

$$ p_A + \sqrt{\frac{2 log(N)}{n_A}} $$  

- $p_A$ is the conversion rate for that version so far  
- where $N$ is the total number of rounds (for all sites) 
- $n_A$ is the number of times site A has been shown

Upper Confidence Bound (UCB)
-----

Proven to have a logarithmic upper bound for regret (hence its name). 

You should first make sure to play each bandit once so none of the $n_A$'s are 0  

Upper Confidence Bound (UCB) Features
-----

* UCB does __not__ use randomness at all. Unlike epsilon-greedy, it’s possible to know exactly how UCB will behave in any given situation. This can make it easier to reason about at times. 

* UCB does __not__ have free parameters that you need to configure before you can deploy it. This is a major improvement if you’re interested in running in the wild, because it means that you can start UCB without having clear sense of how you expect the world to behave 

In [121]:
def ucb1(self):
    """Pick the bandit according to the UCB1 strategy.
    Return the index of the winning bandit."""
    ### BEGIN SOLUTION
    
    # Make sure to play each bandit at least once
    if len(self.trials.nonzero()[0]) < self.n_bandits:
        return np.random.randint(0, self.n_bandits)

    # Compute the UCB1 values for each bandit
    ucb1_vals = np.zeros(len(self.wins))
    for i in range(len(self.wins)):
        ucb1_vals[i] = (self.wins[i] 
                        / (self.trials[i]+1.)
                        + math.sqrt(2.*math.log(self.N) / self.trials[i]))

    # return the max
    return np.argmax(ucb1_vals)
    ### END SOLUTION

Further Study
------

[Boltzmann Exploration Done Right](https://papers.nips.cc/paper/7208-boltzmann-exploration-done-right.pdf)